# ROS workspaces, topics, servicies and actions

## Workspace

Before you start writing any ROS code, you need to set up a workspace for this code to
live in. A workspace is simply a set of directories in which a related set of ROS code
lives. You can have multiple ROS workspaces, but you can only work in one of them
at any one time. The simple way to think about this is that you can only see code that
lives in your current workspace.



Start by making sure that you’ve added the system-wide ROS setup script to
your .bashrc file:


```
user@hostname$ source /opt/ros/indigo/setup.bash
```

Now, we’re going to make a catkin workspace and initialize it:

```
user@hostname$ mkdir -p ~/catkin_ws/src
user@hostname$ cd ~/catkin_ws/src
user@hostname$ catkin_init_workspace
```

This creates a workspace directory called catkin_ws (although you can call it anything
you like), with a src directory inside it for your code. The catkin_init_workspace
command creates a CMakeLists.txt file for you in the src directory, where you invoked
it. 1 Next, we’re going to create some other workspace files:

```
user@hostname$ cd ~/catkin_ws
user@hostname$ catkin_make
```

Running catkin_make will generate a lot of output as it does its work. When it’s done,
you’ll end up with two new directories: build and devel. build is where catkin is going
to store the results of some of its work, like libraries and executable programs if you
use C++. We’ll largely ignore build since we don’t need it much when using Python.
devel contains a number of files and directories, the most interesting of which are the
setup files. Running these configures your system to use this workspace, and the code
that’s (going to be) contained inside it. Assuming you’re using the default command-
line shell ( bash ) and are still in the top-level directory of your workspace, you can do
this with:

```
user@hostname$ source devel/setup.bash
```

Congratulations! You’ve just created your first ROS workspace. You should put all the
code for this book, and any additional code you write that’s based on it, into this
workspace, in the src directory, organized as ROS packages.

## ROS Packages

ROS software is organized into packages, each of which contains some combination
of code, data, and documentation.
Packages sit inside workspaces, in the src directory. Each package directory must
include a CMakeLists.txt file and a package.xml file that describes the contents of the
package and how catkin should interact with it. Creating a new package is easy:

```
user@hostname$ cd ~/catkin_ws/src
user@hostname$ catkin_create_pkg my_awesome_code rospy
```

This changes the directory to src (where packages live) and invokes catkin_cre
ate_pkg to make the new package called my_awesome_code , which depends on the
(already existing) rospy package. 

## roslaunch

oslaunch is a command-line tool designed to automate the launching of collections
of ROS nodes. On the surface, it looks a lot like rosrun , needing a package name and
a filename:

```
user@hostname$ roslaunch PACKAGE LAUNCH_FILE
```

However, roslaunch operates on launch files, rather than nodes. Launch files are
XML files that describe a collection of nodes along with their topic remappings and parameters. By convention, these files have a suffix of `.launch`. For example, here is
`talker_listener.launch` in the rospy_tutorials package:

```xml
<launch>
    <node name="talker" pkg="rospy_tutorials"
        type="talker.py" output="screen" args = "1"/>
    <node name="listener" pkg="rospy_tutorials"
        type="listener.py" output="screen" args = "2"/>
</launch>
```

Each <node> tag includes attributes declaring the ROS graph name of the node, the
package in which it can be found, and the type of node, which is simply the filename
of the executable program. In this example, the output="screen" attributes indicate
that the talker and listener nodes should dump their console outputs to the cur‐
rent console, instead of only to log files. This is a commonly used setting for debug‐
ging; once things start working, it is often convenient to remove this attribute so that
the console has less noise.

## Topics

As we saw in the previous chapter, ROS systems consist of a number of independent
nodes that comprise a graph. These nodes by themselves are typically not very useful.
Things only get interesting when nodes communicate with each other, exchanging
information and data. The most common way to do that is through topics. A topic is a
name for a stream of messages with a defined type. For example, the data from a laser
range-finder might be sent on a topic called scan , with a message type of LaserScan ,
while the data from a camera might be sent over a topic called image , with a message
type of Image .
Topics implement a publish/subscribe communication mechanism, one of the more
common ways to exchange data in a distributed system. Before nodes start to trans‐
mit data over topics, they must first announce, or advertise, both the topic name and
the types of messages that are going to be sent. Then they can start to send, or publish,
the actual data on the topic. Nodes that want to receive messages on a topic can sub‐
scribe to that topic by making a request to roscore . After subscribing, all messages on
the topic are delivered to the node that made the request. One of the main advantages
to using ROS is that all the messy details of setting up the necessary connections
when nodes advertise or subscribe to topics is handled for you by the underlying
communication mechanism so that you don’t have to worry about it yourself.
In ROS, all messages on the same topic must be of the same data type. Although ROS
does not enforce it, topic names often describe the messages that are sent over them.
For example, on the PR2 robot, the topic /wide_stereo/right/image_color is used
for color images from the rightmost camera of the wide-angle stereo pair.

```python
#!/usr/bin/env python
import rospy
from std_msgs.msg import Int32
rospy.init_node('topic_publisher')
pub = rospy.Publisher('counter', Int32)
rate = rospy.Rate(2)
count = 0
while not rospy.is_shutdown():
    pub.publish(count)
    count += 1
    rate.sleep()
```

The first line:

`#!/usr/bin/env python`

is known as the shebang. It lets the operating system know that this is a Python file,
and that it should be passed to the Python interpreter. Since we’re going to be running
the nodes we write as programs, we also have to set execute permissions on them
using the Linux chmod command:

```
user@hostname$ chmod u+x topic_publisher.py
```

Help commands to understand what is going on in the topic:


```
user@hostname$ rostopic echo TOPIC_NAME -n 5
user@hostname$ rostopic hz TOPIC_NAME
user@hostname$ rostopic info TOPIC_NAME
user@hostname$ rostopic find std_msgs/Int32
user@hostname$ rostopic pub TOPIC_NAME std_msgs/Int32 1000000
```

If a topic is marked as **latched**
when it is advertised, subscribers automatically get the last message sent when they
subscribe to the topic. In our map_server example, this means that we only need to
mark it as latched and publish it once. Topics can be marked as latched with the
optional latched argument:

```python
pub = rospy.Publisher('map', nav_msgs/OccupancyGrid, latch=True)
```